# Southern Ocean Composites

In [1]:
import xarray as xr
import numpy as np
import xcdat as xc
import xskillscore as xscore
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
# import cmocean as cmo
from scipy.stats import linregress
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from typing import Tuple, List

from matplotlib.gridspec import GridSpec

## Utils

In [1]:
def get_triangle(tos, latmin: float = -38.75, latmax: float = -1.25, lonmin: float = -178.75, lonmax: float = -71.25, RES: float = 2.5):
    DY = latmax - latmin
    DX = lonmax - lonmin 
    dx = RES*round(DX/DY)
    dy = RES

    # print(f"For each latitude step of {dy} degrees, longitude step is {dx}")

    latcoords = np.arange(latmax, latmin-dy, -dy)
    loncoords = np.arange(lonmin, lonmax+dx, dx)
    lonraw = np.arange(lonmin, lonmax+dx, RES)

    ctos = tos.sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))
    nmodel, _, nlon, ntime = ctos.shape
    # print(ctos)

    for i, clon in enumerate(lonraw):
        j = np.where(clon == loncoords)[0]

        if i == nlon: break

        # print("j prior: ", j)
        if len(j) == 0: 
            j = jold
        else: 
            j = j[0]
             
        # print("j: ", j)
        nlats = int(len(latcoords) - j) # nlats below diag
        # print("nlats: ", nlats)
        ctos[:,:nlats,i,:] = np.full((nmodel, nlats,ntime), np.nan) 
        
        jold = j
    
    return ctos

## Prep Data

In [ ]:
def fix_coords(data):
    data = data.bounds.add_bounds("X")
    data = data.bounds.add_bounds("Y")
    data = data.bounds.add_bounds("T")
    data = xc.swap_lon_axis(data, to=(-180, 180))
    return data

SEB_CMIP6 = xr.open_dataset("data/piControl/SEB_CMIP6_full.nc")*3
SEB_CMIP5 = xr.open_dataset("data/piControl/SEB_CMIP5_full.nc")*3
SEB_CMIP6["NON_CLOUD"] = SEB_CMIP6["SW_Other"] - SEB_CMIP6["LW_Other"]
SEB_CMIP5["NON_CLOUD"] = SEB_CMIP5["SW_Other"] - SEB_CMIP5["LW_Other"]
shared_models_cmip6 = SEB_CMIP6.model.values
shared_models_cmip5 = SEB_CMIP5.model.values
print(SEB_CMIP6)

ERA = "CMIP6"
U10_CMIP6 = xr.open_dataarray(f"data/piControl/rolling_gradient_U10_{ERA}.nc")*3
V10_CMIP6 = xr.open_dataarray(f"data/piControl/rolling_gradient_V10_{ERA}.nc")*3
 
U10_CMIP6 = fix_coords(U10_CMIP6.rename("U10").to_dataset())
V10_CMIP6 = fix_coords(V10_CMIP6.rename("V10").to_dataset())
print(V10_CMIP6)

# Load Rolling Gradient for Regions TOS CMIP6
SST_T_EPSA_CMIP6 = xr.open_dataarray("data/piControl/rolling_gradient_cmip6_eastPacificSA_trend.nc").sel(model=shared_models_cmip6)
SST_T_WE_CMIP6 = xr.open_dataarray("data/piControl/rolling_gradient_cmip6_WE-Trend.nc").sel(model=shared_models_cmip6)
SST_T_E_CMIP6 = xr.open_dataarray("data/piControl/rolling_gradient_cmip6_eastPacific_trend.nc").sel(model=shared_models_cmip6)
SST_T_SO_CMIP6 = xr.open_dataarray("data/piControl/rolling_gradient_cmip6_southernOcean_trend.nc").sel(model=shared_models_cmip6)
SST_T_EPT_CMIP6 = xr.open_dataarray("data/piControl/rolling_gradient_cmip6_eastPacificTriangle_trend.nc").sel(model=shared_models_cmip6)

# Load Rolling Gradient for Regions TOS CMIP5
SST_T_EPSA_CMIP5 = xr.open_dataarray("data/piControl/rolling_gradient_cmip5_eastPacificSA_trend.nc").sel(model=shared_models_cmip5)
SST_T_WE_CMIP5 = xr.open_dataarray("data/piControl/rolling_gradient_cmip5_WE-Trend.nc").sel(model=shared_models_cmip5)
SST_T_E_CMIP5 = xr.open_dataarray("data/piControl/rolling_gradient_cmip5_eastPacific_trend.nc").sel(model=shared_models_cmip5)
SST_T_SO_CMIP5 = xr.open_dataarray("data/piControl/rolling_gradient_cmip5_southernOcean_trend.nc").sel(model=shared_models_cmip5)
SST_T_EPT_CMIP5 = xr.open_dataarray("data/piControl/rolling_gradient_cmip5_eastPacificTriangle_trend.nc").sel(model=shared_models_cmip5)

def get_shared_models(ds1: xr.Dataset, ds2: xr.Dataset) -> Tuple[xr.Dataset, xr.Dataset]:
    shared_models = list(set(ds1.model.values).intersection(set(ds2.model.values)))
    return ds1.sel(model=shared_models), ds2.sel(model=shared_models)
    
SWCF_CMIP6 = xr.open_dataarray("data/piControl/swcf_east_sa_cmip6_v3.nc")
SWCF_CMIP5 = xr.open_dataarray("data/piControl/swcf_east_sa_cmip5_v3.nc")

SWCF_CMIP6, SEB_CMIP6 = get_shared_models(SWCF_CMIP6, SEB_CMIP6)
SWCF_CMIP5, SEB_CMIP5 = get_shared_models(SWCF_CMIP5, SEB_CMIP5)
SEB_CMIP6, SST_T_EPT_CMIP6 = get_shared_models(SEB_CMIP6, SST_T_EPT_CMIP6)
SEB_CMIP5, SST_T_EPT_CMIP5 = get_shared_models(SEB_CMIP5, SST_T_EPT_CMIP5)

U10_CMIP6, SEB_CMIP6 = get_shared_models(U10_CMIP6, SEB_CMIP6)
V10_CMIP6, SEB_CMIP6 = get_shared_models(V10_CMIP6, SEB_CMIP6)

SWCF_CMIP6, SEB_CMIP6 = get_shared_models(SWCF_CMIP6, SEB_CMIP6)
SWCF_CMIP5, SEB_CMIP5 = get_shared_models(SWCF_CMIP5, SEB_CMIP5)

In [ ]:
SEB_CMIP6_SO = SEB_CMIP6.sel(lon=slice(-180, -75), lat=slice(-70, -50)).spatial.average("NET")["NET"]
SO_Thres =  SEB_CMIP6_SO.mean("time") - SEB_CMIP6_SO.std("time")
SEB_SO_1sig = SEB_CMIP6.where(SEB_CMIP6_SO <= SO_Thres).mean(["time", "model"])
U10_SO_1sig = U10_CMIP6.where(SEB_CMIP6_SO <= SO_Thres).mean(["time", "model"])
V10_SO_1sig = V10_CMIP6.where(SEB_CMIP6_SO <= SO_Thres).mean(["time", "model"])